# Tutorial for SQL Langchain
https://python.langchain.com/en/latest/modules/chains/examples/sqlite.html
https://www.youtube.com/watch?v=ylWAxc0Mrjc

In [1]:
# Requirements
%pip install langchain watermark openai

                                              0.0/949.7 kB ? eta -:--:--
     ----                                   112.6/949.7 kB 3.3 MB/s eta 0:00:01
     -----------                            286.7/949.7 kB 3.5 MB/s eta 0:00:01
     ----------------------                 553.0/949.7 kB 4.3 MB/s eta 0:00:01
     -----------------------                583.7/949.7 kB 3.7 MB/s eta 0:00:01
     -------------------------------------- 949.7/949.7 kB 5.5 MB/s eta 0:00:00
  Using cached PyYAML-6.0-cp311-cp311-win_amd64.whl (143 kB)
                                              0.0/2.0 MB ? eta -:--:--
     --------                                 0.4/2.0 MB 12.5 MB/s eta 0:00:01
     -----------------                        0.8/2.0 MB 10.6 MB/s eta 0:00:01
     ---------------------------              1.4/2.0 MB 10.7 MB/s eta 0:00:01
     --------------------------------------   1.9/2.0 MB 10.9 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 10.4 MB/s eta 0:00:00


In [3]:
import os
import openai
import warnings
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

In [14]:
# Use the schema to create sql code to create a table
load_dotenv()
openai.api_key = os.getenv("OPENAI_API")

os.getenv("OPENAI_API_KEY")

In [6]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain


In [19]:
sqlite_db_path = "tutorial/chinook.db"
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")
llm = OpenAI(temperature=0)

In [20]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [21]:
db_chain.run("How many employees are there?")



> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM employees;
SQLResult: [(8,)]
Answer:There are 8 employees.
> Finished chain.


'There are 8 employees.'

In [22]:
db_chain_checked = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)
db_chain_checked.run("How many employees are there and what are their first names?")



> Entering new SQLDatabaseChain chain...
How many employees are there and what are their first names?
SQLQuery:SELECT COUNT(*), "FirstName" FROM employees LIMIT 5;
SQLResult: [(8, 'Andrew')]
Answer:There are 8 employees and their first names are Andrew.
> Finished chain.


'There are 8 employees and their first names are Andrew.'

# Customize Prompt
You can also customize the prompt that is used. Here is an example prompting it to understand that foobar is the same as the Employee table

In [23]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone asks for the table foobar, they really mean the employee table.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

In [24]:
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)
db_chain.run("How many employees are there in the foobar table?")



> Entering new SQLDatabaseChain chain...
How many employees are there in the foobar table?
SQLQuery:SELECT COUNT(*) FROM employees;
SQLResult: [(8,)]
Answer:There are 8 employees in the foobar table.
> Finished chain.


'There are 8 employees in the foobar table.'